In [1]:
%load_ext autoreload
%autoreload 2

# Extract features, retrain Sherlock and generate predictions.

The script below first downloads the data (roughly 700K samples), then extract features from the raw data values. <br>
If you want to skip this step, you can follow the steps below the feature extraction to load the preprocessed data, 
retrain Sherlock and generate predictions.

In [2]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import tensorflow as tf
from ast import literal_eval

from sherlock import helpers
from sherlock.features.preprocessing import extract_features, convert_string_lists_to_lists, prepare_feature_extraction
from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock

from pympler import muppy, summary
from datetime import datetime

## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [3]:
helpers.download_data()
prepare_feature_extraction()

Data was downloaded.
Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


## Read in raw data
You can skip this step if you want to use a preprocessed data file.

## Report memory usage (can be slow)

In [4]:
report_memory = False

## Extract features
It is important that the string-representations of lists are first converted into lists of strings.
The labels should be a list of semantic types.

In [5]:
import time 

timestr = time.strftime("%Y%m%d-%H%M%S")

X_test_filename_csv = f'../data/work/test_{timestr}.csv'
X_train_filename_csv = f'../data/work/train_{timestr}.csv'
X_validation_filename_csv = f'../data/work/validation_{timestr}.csv'

### PREPARATION

In [6]:
# ensure embedding initialisation is outside of timing for extract_features
from sherlock.features.word_embeddings import initialise_word_embeddings
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import prepare_feature_extraction

prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:06.035376 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:02.502077 seconds. (filename = ../sherlock/features/par_vec_retrained_400.pkl)
Initialised NLTK, process took 0:00:00.310960 seconds.


[nltk_data] Downloading package punkt to /Users/lowecg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lowecg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from pyarrow.parquet import ParquetFile

def load_parquet_values(path):
    pf = ParquetFile(source=path)
    row_df = pf.read_row_group(0)

    return row_df['values']

In [8]:
%load_ext line_profiler

## EXTRACT FEATURES TO CSV (NEW METHOD)

### TEST SET

In [9]:
# from sherlock.functional import extract_features_to_csv2
# from sherlock.global_state import reset_first

# reset_first()

# values = load_parquet_values("../data/raw/test_values.parquet")

# %lprun -m sherlock.features.preprocessing -m sherlock.functional -m sherlock.features extract_features_to_csv2('deleteme.csv', values, 100)

# values = None

from sherlock.functional import extract_features_to_csv

values = load_parquet_values("../data/raw/test_values.parquet")

extract_features_to_csv(X_test_filename_csv, values)

values = None

Starting ../data/work/test_20210107-072852.csv at 2021-01-07 07:29:02.762640
Exporting 1578 column features
Finished. Processed 137353 rows in 0:04:56.936439, key_count=8


In [10]:
# Finished. Processed 137353 rows in 0:14:53.196073

In [11]:
print(f'Finished at {datetime.now()}')

Finished at 2021-01-07 07:33:59.881479


### TRAIN SET

In [ ]:
values = load_parquet_values("../data/raw/train_values.parquet")

extract_features_to_csv(X_train_filename_csv, values)

values = None

Starting ../data/work/train_20210107-072852.csv at 2021-01-07 07:34:02.583582
Exporting 1578 column features


In [ ]:
print(f'Finished at {datetime.now()}')

### VALIDATION SET

In [ ]:
values = load_parquet_values("../data/raw/val_values.parquet")

extract_features_to_csv(X_validation_filename_csv, values)

values = None

In [ ]:
print(f'Finished at {datetime.now()}')

### Read Locally Processed Features

In [ ]:
start = datetime.now()

X_test = pd.read_csv(X_test_filename_csv, dtype=np.float32)

print(f'Load Features (test) process took {datetime.now() - start} seconds.')

In [ ]:
X_test.head()

In [ ]:
start = datetime.now()

X_train = pd.read_csv(X_train_filename_csv, dtype=np.float32)

print(f'Load Features (train) process took {datetime.now() - start} seconds.')

In [ ]:
X_train.head()

In [ ]:
start = datetime.now()

X_validation = pd.read_csv(X_validation_filename_csv, dtype=np.float32)

print(f'Load Features (validation) process took {datetime.now() - start} seconds.')

In [ ]:
X_validation.head()

## Impute NaN values with feature means

In [ ]:
start = datetime.now()

train_columns_means = pd.DataFrame(X_train.mean()).transpose()

print(f'Transpose process took {datetime.now() - start} seconds.')

In [ ]:
start = datetime.now()

X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_validation.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)

train_columns_means=None

print(f'FillNA process took {datetime.now() - start} seconds.')

In [ ]:
start = datetime.now()

X_train.to_parquet('train.parquet', engine='pyarrow', compression='snappy')
X_validation.to_parquet('validation.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('test.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process took {datetime.now() - start} seconds.')

## EXTRACT FEATURES TO CSV (**OLD** METHOD) 

### TEST SET (OLD METHOD)

In [ ]:
if report_memory:
    all_objects = muppy.get_objects()
    sum1 = summary.summarize(all_objects)
    # Prints out a summary of the large objects
    summary.print_(sum1)

In [ ]:
test_samples = pd.read_parquet('../data/raw/test_values.parquet')
test_labels = pd.read_parquet('../data/raw/test_labels.parquet')

In [ ]:

test_samples_converted, y_test = convert_string_lists_to_lists(test_samples, test_labels, "values", "type")

In [ ]:
# free memory
test_samples = None
test_labels = None

In [ ]:
test_samples_converted.head()

In [ ]:
# output "head" 
y_test[:5]

### Given that feature extraction can take long, we only take the first 100 samples.

In [ ]:
y_test_subset = y_test[:100]

In [ ]:
if report_memory:
    all_objects = muppy.get_objects()
    sum1 = summary.summarize(all_objects)
    # Prints out a summary of the large objects
    summary.print_(sum1)

In [ ]:
from sherlock.features.preprocessing import extract_features

start = datetime.now()

#extract_features(X_test_filename, test_samples_converted.head(n=100))

%lprun -m sherlock.features.preprocessing X_test=extract_features('deleteme.csv', test_samples_converted.head(n=100)) 

print(f'Extract Features (test) process took {datetime.now() - start} seconds.')

In [ ]:
test_samples_converted = None

In [ ]:
from sherlock.features.helpers import generate_chars_col
generate_chars_col()

In [ ]:
import gc

gc.collect()

In [ ]:
if report_memory:
    all_objects = muppy.get_objects()
    sum1 = summary.summarize(all_objects)
    # Prints out a summary of the large objects
    summary.print_(sum1)

In [ ]:
# over all, without memory management
# Extract Features (test) process took 3:40:25.799880 seconds.

In [ ]:
# Baseline
# Extract Features (test) process took 0:11:04.137081 seconds.

# Iterations
# Extract Features (test) process took 0:00:56.671353 seconds. (cache word embeddings)
# Extract Features (test) process took 0:00:13.523261 seconds. (cache Doc2Vec)
# Extract Features (test) process took 0:00:03.674007 seconds. (loads of tweaks, use np.array for stats)
# Extract Features (test) process took 0:00:03.262298 seconds. (manually compute counts)
# Extract Features (test) process took 0:00:02.853031 seconds. (replace series.str.count in BoW, use series.tolist() in paragraph vectors )
# Extract Features (test) process took 0:00:01.718025 seconds. (compute mean, variance, skew and kurtosis together)
# Extract Features (test) process took 0:00:01.437484 seconds. (compute sum, min and max together, nunique replaced with len(set(series))), use statistics.median not np.median)

### TRAIN SET (OLD METHOD)

In [ ]:
train_samples = pd.read_parquet('../data/raw/train_values.parquet')
train_labels = pd.read_parquet('../data/raw/train_labels.parquet')

In [ ]:
train_samples_converted, y_train = convert_string_lists_to_lists(train_samples, train_labels, "values", "type")

In [ ]:
# free memory
train_samples = None
train_labels = None

In [ ]:
y_train_subset = y_train[:100]

In [ ]:
start = datetime.now()

extract_features(X_train_filename, train_samples_converted)

print(f'Extract Features (train) process took {datetime.now() - start} seconds.')

In [ ]:
train_samples_converted = None

### VALIDATION SET (OLD METHOD)

In [ ]:
validation_samples = pd.read_parquet('../data/raw/val_values.parquet')
validation_labels = pd.read_parquet('../data/raw/val_labels.parquet')

In [ ]:
validation_samples_converted, y_validation = convert_string_lists_to_lists(validation_samples, validation_labels, "values", "type")

In [ ]:
# free memory
validation_samples = None
validation_labels = None

In [ ]:
y_validation_subset = y_validation[:100]

In [ ]:
start = datetime.now()

extract_features(X_validation_filename, validation_samples_converted)

print(f'Extract Features (validation) process took {datetime.now() - start} seconds.')

In [ ]:
validation_samples_converted = None